In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

colors2 = px.colors.qualitative.Set2
colors1 = px.colors.qualitative.Dark2
symbols = ['circle', 'x', 'hexagram', 'star-diamond', 'cross', 'y-up', 'asterisk', 'hexagon']

def generate_style():
    lst = []
    for i in range(0, 8, 1):
        lst.append(dict(color=colors1[i], symbol = symbols[i], opacity=1., line=dict(color='DarkSlateGrey', width=1)))
    return lst

marker_styles = generate_style()

In [10]:
rv = 0

fig = go.Figure()
for rv in [3]:
    for j, R in enumerate([1/2, 1/3, 1/4, 1/5]):
        data = pd.read_table('data/results_MSA_{:.3f}_{}.txt'.format(R, rv), names = ['SNR', 'rv', 'R', 'BER', 'BERinf', 'FSR', 'FSRinf'])
        data = data.query('SNR <= 7')
        data = data.groupby(['SNR']).agg([np.mean]).reset_index()


        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = np.array(data.BER['mean']),
                                mode = 'lines+markers',
                                visible = 'legendonly',
                                legendgroup='BER',
                                line = dict(dash = 'dot', color = colors2[4 + j]),
                                name = 'BER (all), R = {:.3f}, rv = {}'.format(R, rv)))

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = np.array(data.BERinf['mean']),
                                mode = 'lines+markers',
                                #visible = 'legendonly',
                                #marker = marker_styles[j],
                                legendgroup='BER inf',
                                line = dict(dash = 'dot', color = colors2[j]),
                                name = 'BER (data), R = {:.3f}, rv = {}'.format(R, rv)))

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = 1 - np.array(data.FSR['mean']),
                                mode = 'lines+markers',
                                visible = 'legendonly',
                                marker = marker_styles[j + 4],
                                legendgroup='FER',
                                name = 'FER (all), R = {:.3f}, rv = {}'.format(R, rv)))

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = 1 - np.array(data.FSRinf['mean']),
                                mode = 'lines+markers',
                                marker = marker_styles[j],
                                #line = dict(dash = 'dot'),
                                visible = 'legendonly',
                                legendgroup='FER inf',
                                name = 'FER (data), R = {:.3f}, rv = {}'.format(R, rv)))


fig.update_layout(title='MSA',
                   xaxis_title='SNR, dB')
fig.update_yaxes(type="log")
fig.update_xaxes(range=[-5, 7])

fig.show()
fig.write_html("MSA_rates.html")

In [9]:
from plotly.subplots import make_subplots
fig = go.Figure()
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{}, {}],
           [{}, {}]],
    print_grid=True,
    subplot_titles = ['R = 0.500', 'R = 0.333', 'R = 0.250', 'R = 0.200'])

rows_n = [1, 1, 2, 2]
cols_n = [1, 2, 1, 2]
show = [True, False, False, False]

for i, R in enumerate([1/2, 1/3, 1/4, 1/5]):
    for j, rv in enumerate([0, 1, 2, 3]): 
        data = pd.read_table('data/results_MSA_{:.3f}_{}.txt'.format(R, rv), names = ['SNR', 'rv', 'R', 'BER', 'BERinf', 'FSR', 'FSRinf'])
        data = data.query('SNR <= 7')
        data = data.groupby(['SNR']).agg([np.mean]).reset_index()


        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = np.array(data.BER['mean']),
                                mode = 'lines+markers',
                                visible = 'legendonly',
                                legendgroup='BER',
                                showlegend = show[i],
                                line = dict(dash = 'dot', color = colors2[4 + j]),
                                name = 'BER (all), rv = {}'.format(rv)), row = rows_n[i], col = cols_n[i])

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = np.array(data.BERinf['mean']),
                                mode = 'lines+markers',
                                legendgroup='BER inf',
                                showlegend = show[i],
                                line = dict(dash = 'dot', color = colors2[j]),
                                name = 'BER (data), rv = {}'.format(rv)), row = rows_n[i], col = cols_n[i])

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = 1 - np.array(data.FSR['mean']),
                                mode = 'lines+markers',
                                visible = 'legendonly',
                                marker = marker_styles[j + 4],
                                legendgroup='FER',
                                showlegend = show[i],
                                name = 'FER (all), rv = {}'.format(rv)), row = rows_n[i], col = cols_n[i])

        fig.add_trace(go.Scatter(x = np.array(data.SNR), y = 1 - np.array(data.FSRinf['mean']),
                                mode = 'lines+markers',
                                marker = marker_styles[j],
                                legendgroup='FER inf',
                                showlegend = show[i],
                                visible = 'legendonly',
                                name = 'FER (data), rv = {}'.format(rv)), row = rows_n[i], col = cols_n[i])

        fig.update_yaxes(type="log", row=rows_n[i], col=cols_n[i])
        fig.update_xaxes(title_text='SNR, dB', range=[-5, 7])

fig.update_layout(title='MSA')
fig.update_layout(height=900, width=1600)
fig.write_html("rv.html")
fig.show()


This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

